# Dependency Graphs

This is a notebook to experiment with a set of dependency graphs. These graphs have been created from Python applications and they reflect graphs with different versions of package. We call these graphs of versioned packages a 'stack'.

First of all we will include all the required modules and configure matplotlib to draw diagrams inlined in this notebook.

In [ ]:
import json

import networkx as nx
from networkx.algorithms.operators.binary import compose
from networkx.readwrite import json_graph
import matplotlib.pyplot as plt

%matplotlib inline

Now we define a function that will read the output of `pipenv graph --json` from a file and create the equivalent graph using `networkx`.

In [ ]:
def graph_from_file(filename):
    data = None
    G = None
    
    with open(filename) as file:    
        data = json.load(file)
        
    if data is not None:
        G = nx.DiGraph()
        for package in data:
            _node_id = '{}-{}'.format(package['package']['key'], package['package']['installed_version'])
            G.add_node(_node_id, name=package['package']['package_name'], version=package['package']['installed_version'])
            
            for dependency in package['dependencies']:
                _dependency_node_id = '{}-{}'.format(dependency['key'], dependency['installed_version'])
                G.add_node(_dependency_node_id, name=dependency['package_name'], version=dependency['installed_version'])
                G.add_edge(_node_id, _dependency_node_id, relation='depends_on')
                
    return G

We will read a set of files from the `fixtures/` directory:

In [ ]:
PIPENV_GRAPH_FILES = ['fixtures/tensorflow-1.4.0.json', 'fixtures/tensorflow-1.4.1.json', 
                      'fixtures/tensorflow-1.3.0.json', 'fixtures/keras-2.1.2.json', 
                      'fixtures/keras-2.1.2-tensorflow-1.3.0.json']

Each file will be read and added to our big huge global graph G.

In [ ]:
G = nx.DiGraph()

for filename in PIPENV_GRAPH_FILES:
    _H = graph_from_file(filename)
    G = compose(G, _H)


The next cell will draw the whole graph G, node labels are added. 

In [ ]:
plt.figure(figsize=(16, 9))
plt.axis('off')

pos = nx.spring_layout(G, iterations=20)
labels=dict((n,d['name']) for n,d in G.nodes(data=True))

nx.draw_networkx_nodes(G, pos, labels=labels, node_size=40, node_color='blue')
nx.draw_networkx_edges(G, pos, alpha=0.2)
nx.draw_networkx_labels(G, pos, fontsize=12)


... and the next cell is just for debugging...

In [ ]:
dataH = json_graph.node_link_data(G)

json.dumps(dataH)